<h1>Advanved Stock Analysis with Python</h1>

<h3>Web Scraping</h3>

If you look at quotes for various securities on Yahoo Finance, you might notice that the URL is always in the following format ```https://finance.yahoo.com/quote/{symbol}```. We can use this knowledge to quickly collect information for any security on Yahoo Finance.

The code below shows how you can concatenate a URL with Python code.

In [1]:
ticker = 'MSFT'
url = f'https://finance.yahoo.com/quote/{ticker}'
print(url)

https://finance.yahoo.com/quote/MSFT


In [2]:
ticker = 'TSLA'
url = f'https://finance.yahoo.com/quote/{ticker}'
print(url)

https://finance.yahoo.com/quote/TSLA


Now, to collect data from Yahoo Finance, we need to import a couple of libraries that give us access to various methods (or commands) that we want to use to collect the information from our desired URLs.

In [3]:
#Import libraries
import requests
from bs4 import BeautifulSoup

The first thing we need to do when using Python to collect data from a website is to perform a GET request on the URL.

To check whether or not the GET request was successful, we can return the status code of the get request.

If the status code is 200, that means that the GET request was successful.

In [4]:
#headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"}
r = requests.get(url)
print(r.status_code)

200


At this point, we will use Beautiful Soup to parse the data from a particular HTML element titled "D(ib) Mend(20px)" on Yahoo Finance, which will represent the stock price from the URL for our security.

In [5]:
soup = BeautifulSoup(r.text, 'html.parser')
data = soup.find('div', {'class': 'D(ib) Mend(20px)'}).find_all('fin-streamer')
data

[<fin-streamer active="" class="Fw(b) Fz(36px) Mb(-4px) D(ib)" data-field="regularMarketPrice" data-pricehint="2" data-symbol="TSLA" data-test="qsp-price" data-trend="none" value="905.39">905.39</fin-streamer>,
 <fin-streamer active="" class="Fw(500) Pstart(8px) Fz(24px)" data-field="regularMarketChange" data-pricehint="2" data-symbol="TSLA" data-test="qsp-price-change" data-trend="txt" value="33.79004"><span class="C($positiveColor)">+33.79</span></fin-streamer>,
 <fin-streamer active="" class="Fw(500) Pstart(8px) Fz(24px)" data-field="regularMarketChangePercent" data-pricehint="2" data-symbol="TSLA" data-template="({fmt})" data-trend="txt" value="0.03876783"><span class="C($positiveColor)">(+3.88%)</span></fin-streamer>,
 <fin-streamer active="true" changeev="regularTimeChange" class="D(n)" data-field="regularMarketTime" data-symbol="TSLA" data-trend="none" value=""></fin-streamer>,
 <fin-streamer active="true" changeev="marketState" class="D(n)" data-field="marketState" data-symbol=

At this point, the data looks pretty messy, but we can use ```.text``` to find the information between the `> <` symbols, which represents the information displayed on the website.

With a little bit of Python code, we can extract all of the information we are interested in.

In [6]:
for i in range(len(data)):
    print(data[i].text)

905.39
+33.79
(+3.88%)




Now, let's combine everything we learned above to create a function that can get the data for any security.

In [7]:
def getData(symbol):
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"}
    url = f'https://finance.yahoo.com/quote/{symbol}'

    r = requests.get(url, headers=headers)
    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'html.parser')

        stock = {
            'symbol': symbol,
            'price' : soup.find('div', {'class': 'D(ib) Mend(20px)'}).find_all('fin-streamer')[0].text,
            'change' : soup.find('div', {'class': 'D(ib) Mend(20px)'}).find_all('fin-streamer')[1].text,
            'percent change': soup.find('div', {'class': 'D(ib) Mend(20px)'}).find_all('fin-streamer')[2].text
        }
        return stock
    else:
        return ("Error:", r.status_code)
    

This function works for any symbol that exists on yahoo finance, as shown below:

In [8]:
getData("MSFT")

{'symbol': 'MSFT',
 'price': '300.43',
 'change': '+5.21',
 'percent change': '(+1.76%)'}

In [9]:
getData("TSLA")

{'symbol': 'TSLA',
 'price': '905.39',
 'change': '+33.79',
 'percent change': '(+3.88%)'}

In [10]:
getData("BTC-USD")

{'symbol': 'BTC-USD',
 'price': '41,356.23',
 'change': '-639.61',
 'percent change': '(-1.52%)'}

<h3>YFinance Library</h3>

For Yahoo Finance, we can use the yfinance library which does most of the hard work for us, so we can focus more on analyzing data than collecting it.

In [11]:
#Import libraries and install yfinance (if needed)
#%pip install yfinance
import yfinance as yf
import pandas as pd

Now, to collect everything we want on Tesla, for example, we just pass "TSLA" into the ```yf.Ticker()``` function, and store it into a new variable.

In [12]:
tsla = yf.Ticker("TSLA")

We can now call ```tsla.info``` to get almost everything available about Tesla on Yahoo Finance. There is so much information available just by calling ```tsla.info```:

In [13]:
tsla.info

{'zip': '78725',
 'sector': 'Consumer Cyclical',
 'fullTimeEmployees': 99290,
 'longBusinessSummary': 'Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, and internationally. The company operates in two segments, Automotive, and Energy Generation and Storage. The Automotive segment offers electric vehicles, as well as sells automotive regulatory credits. It provides sedans and sport utility vehicles through direct and used vehicle sales, a network of Tesla Superchargers, and in-app upgrades; and purchase financing and leasing services. This segment is also involved in the provision of non-warranty after-sales vehicle services, sale of used vehicles, retail merchandise, and vehicle insurance, as well as sale of products to third party customers; services for electric vehicles through its company-owned service locations, and Tesla mobile service technicians; and vehicle limited warrantie

Here are a few things that we found:

In [14]:
print("Current price:", tsla.info['currentPrice'])
print("Current ratio:", tsla.info['currentRatio'])
print("Debt to equity ratio:", tsla.info['debtToEquity'])
print("Quick ratio:", tsla.info['quickRatio'])
print("Beta:", tsla.info['beta'])


Current price: 905.39
Current ratio: 1.375
Debt to equity ratio: 28.192
Quick ratio: 0.999
Beta: 2.04872


We can also use Python to generate a table containing the price history of Tesla.

In [15]:
tsla.history(period="max")

Open        High         Low       Close    Volume  \
Date                                                                   
2010-06-29    3.800000    5.000000    3.508000    4.778000  93831500   
2010-06-30    5.158000    6.084000    4.660000    4.766000  85935500   
2010-07-01    5.000000    5.184000    4.054000    4.392000  41094000   
2010-07-02    4.600000    4.620000    3.742000    3.840000  25699000   
2010-07-06    4.000000    4.000000    3.166000    3.222000  34334500   
...                ...         ...         ...         ...       ...   
2022-03-14  780.609985  800.700012  756.039978  766.369995  23717400   
2022-03-15  775.270020  805.570007  756.570007  801.890015  22280400   
2022-03-16  809.000000  842.000000  802.260010  840.229980  28009600   
2022-03-17  830.989990  875.000000  825.719971  871.599976  22194300   
2022-03-18  874.489990  907.849976  867.390015  905.390015  33408500   

            Dividends  Stock Splits  
Date                                 
2010-06-29          0           0.0  
2010-06-30          0           0.0  
2010-07-01          0           0.0  
2010-07-02          0           0.0  
2010-07-06          0           0.0  
...               ...           ...  
2022-03-14          0           0.0  
2022-03-15          0           0.0  
2022-03-16          0           0.0  
2022-03-17          0           0.0  
2022-03-18          0           0.0  

[2952 rows x 7 columns]

If we only want the price history for the last month, we can change the period to `1mo`:

In [16]:
tsla.history(period="1mo")

Open        High         Low       Close    Volume  \
Date                                                                   
2022-02-22  834.130005  856.729980  801.099976  821.530029  27762700   
2022-02-23  830.429993  835.299988  760.559998  764.039978  31752300   
2022-02-24  700.390015  802.479980  700.000000  800.770020  45107400   
2022-02-25  809.229980  819.500000  782.400024  809.869995  25355900   
2022-02-28  815.010010  876.859985  814.710022  870.429993  33002300   
2022-03-01  869.679993  889.880005  853.780029  864.369995  24922300   
2022-03-02  872.130005  886.479980  844.270020  879.890015  24881100   
2022-03-03  878.770020  886.440002  832.599976  839.289978  20541200   
2022-03-04  849.099976  855.650024  825.159973  838.289978  22333200   
2022-03-07  856.299988  866.140015  804.570007  804.580017  24164700   
2022-03-08  795.530029  849.989990  782.169983  824.400024  26799700   
2022-03-09  839.479980  860.559998  832.010010  858.969971  19728000   
2022-03-10  851.450012  854.450012  810.359985  838.299988  19549500   
2022-03-11  840.200012  843.799988  793.770020  795.349976  22272800   
2022-03-14  780.609985  800.700012  756.039978  766.369995  23717400   
2022-03-15  775.270020  805.570007  756.570007  801.890015  22280400   
2022-03-16  809.000000  842.000000  802.260010  840.229980  28009600   
2022-03-17  830.989990  875.000000  825.719971  871.599976  22194300   
2022-03-18  874.489990  907.849976  867.390015  905.390015  33408500   

            Dividends  Stock Splits  
Date                                 
2022-02-22          0             0  
2022-02-23          0             0  
2022-02-24          0             0  
2022-02-25          0             0  
2022-02-28          0             0  
2022-03-01          0             0  
2022-03-02          0             0  
2022-03-03          0             0  
2022-03-04          0             0  
2022-03-07          0             0  
2022-03-08          0             0  
2022-03-09          0             0  
2022-03-10          0             0  
2022-03-11          0             0  
2022-03-14          0             0  
2022-03-15          0             0  
2022-03-16          0             0  
2022-03-17          0             0  
2022-03-18          0             0

We can also show financial statements (both annual and quarterly):

In [17]:
tsla.balance_sheet

2021-12-31    2020-12-31    2019-12-31  \
Intangible Assets          2.570000e+08  3.130000e+08  3.390000e+08   
Capital Surplus            2.980300e+10  2.726000e+10  1.273600e+10   
Total Liab                 3.054800e+10  2.846900e+10  2.619900e+10   
Total Stockholder Equity   3.018900e+10  2.222500e+10  6.618000e+09   
Minority Interest          1.394000e+09  1.454000e+09  1.492000e+09   
Other Current Liab         4.756000e+09  4.147000e+09  3.693000e+09   
Total Assets               6.213100e+10  5.214800e+10  3.430900e+10   
Common Stock               1.000000e+06  1.000000e+06  1.000000e+06   
Other Current Assets       3.450000e+08  2.380000e+08  2.460000e+08   
Retained Earnings          3.310000e+08 -5.399000e+09 -6.083000e+09   
Other Liab                 3.896000e+09  3.302000e+09  2.969000e+09   
Good Will                  2.000000e+08  2.070000e+08  1.980000e+08   
Treasury Stock             5.400000e+07  3.630000e+08 -3.600000e+07   
Other Assets               3.398000e+09  1.536000e+09  1.469000e+09   
Cash                       1.757600e+10  1.938400e+10  6.268000e+09   
Total Current Liabilities  1.970500e+10  1.424800e+10  1.066700e+10   
Short Long Term Debt       1.088000e+09  1.758000e+09  1.399000e+09   
Other Stockholder Equity   5.400000e+07  3.630000e+08 -3.600000e+07   
Property Plant Equipment   3.117600e+10  2.337500e+10  2.019900e+10   
Total Current Assets       2.710000e+10  2.671700e+10  1.210300e+10   
Net Tangible Assets        2.973200e+10  2.170500e+10  6.081000e+09   
Short Term Investments     1.310000e+08           NaN           NaN   
Net Receivables            1.986000e+09  1.903000e+09  1.324000e+09   
Long Term Debt             4.285000e+09  8.571000e+09  1.037500e+10   
Inventory                  5.757000e+09  4.101000e+09  3.552000e+09   
Accounts Payable           1.002500e+10  6.051000e+09  3.771000e+09   
Long Term Investments               NaN           NaN  1.000000e+06   

                             2018-12-31  
Intangible Assets          2.820000e+08  
Capital Surplus            1.024900e+10  
Total Liab                 2.342700e+10  
Total Stockholder Equity   4.923000e+09  
Minority Interest          1.390000e+09  
Other Current Liab         2.955000e+09  
Total Assets               2.974000e+10  
Common Stock                        NaN  
Other Current Assets       1.930000e+08  
Retained Earnings         -5.318000e+09  
Other Liab                 2.318000e+09  
Good Will                  6.800000e+07  
Treasury Stock            -8.000000e+06  
Other Assets               1.380000e+09  
Cash                       3.686000e+09  
Total Current Liabilities  9.993000e+09  
Short Long Term Debt       2.284000e+09  
Other Stockholder Equity  -8.000000e+06  
Property Plant Equipment   1.969100e+10  
Total Current Assets       8.307000e+09  
Net Tangible Assets        4.573000e+09  
Short Term Investments              NaN  
Net Receivables            9.490000e+08  
Long Term Debt             8.461000e+09  
Inventory                  3.113000e+09  
Accounts Payable           3.405000e+09  
Long Term Investments      1.200000e+07

In [18]:
tsla.quarterly_balance_sheet

2021-12-31    2021-09-30    2021-06-30  \
Intangible Assets          2.570000e+08  2.690000e+08  2.830000e+08   
Capital Surplus            2.980300e+10  2.892200e+10  2.820500e+10   
Total Liab                 3.054800e+10  2.934000e+10  2.889600e+10   
Total Stockholder Equity   3.018900e+10  2.705300e+10  2.480400e+10   
Minority Interest          1.394000e+09  1.441000e+09  1.446000e+09   
Other Current Liab         4.756000e+09  5.024000e+09  4.621000e+09   
Total Assets               6.213100e+10  5.783400e+10  5.514600e+10   
Common Stock               1.000000e+06  1.000000e+06  1.000000e+06   
Other Current Assets       3.450000e+08  3.270000e+08  3.260000e+08   
Retained Earnings          3.310000e+08 -1.990000e+09 -3.608000e+09   
Other Liab                 3.896000e+09  3.190000e+09  3.210000e+09   
Good Will                  2.000000e+08  2.010000e+08  2.030000e+08   
Treasury Stock             5.400000e+07  1.200000e+08  2.060000e+08   
Other Assets               3.398000e+09  3.114000e+09  2.937000e+09   
Cash                       1.757600e+10  1.606500e+10  1.622900e+10   
Total Current Liabilities  1.970500e+10  1.805100e+10  1.637100e+10   
Short Long Term Debt       1.088000e+09  1.207000e+09  1.082000e+09   
Other Stockholder Equity   5.400000e+07  1.200000e+08  2.060000e+08   
Property Plant Equipment   3.117600e+10  2.924800e+10  2.703000e+10   
Total Current Assets       2.710000e+10  2.500200e+10  2.469300e+10   
Net Tangible Assets        2.973200e+10  2.658300e+10  2.431800e+10   
Short Term Investments     1.310000e+08  3.000000e+07           NaN   
Net Receivables            1.986000e+09  2.000000e+09  2.159000e+09   
Long Term Debt             4.285000e+09  5.471000e+09  6.915000e+09   
Inventory                  5.757000e+09  5.199000e+09  4.733000e+09   
Accounts Payable           1.002500e+10  8.260000e+09  7.558000e+09   

                             2021-03-31  
Intangible Assets          2.990000e+08  
Capital Surplus            2.762300e+10  
Total Liab                 2.850700e+10  
Total Stockholder Equity   2.301700e+10  
Minority Interest          1.448000e+09  
Other Current Liab         4.412000e+09  
Total Assets               5.297200e+10  
Common Stock               1.000000e+06  
Other Current Assets       3.050000e+08  
Retained Earnings         -4.750000e+09  
Other Liab                 3.247000e+09  
Good Will                  2.060000e+08  
Treasury Stock             1.430000e+08  
Other Assets               2.918000e+09  
Cash                       1.714100e+10  
Total Current Liabilities  1.487700e+10  
Short Long Term Debt       1.415000e+09  
Other Stockholder Equity   1.430000e+08  
Property Plant Equipment   2.484400e+10  
Total Current Assets       2.470500e+10  
Net Tangible Assets        2.251200e+10  
Short Term Investments              NaN  
Net Receivables            1.913000e+09  
Long Term Debt             8.019000e+09  
Inventory                  4.132000e+09  
Accounts Payable           6.648000e+09

In [19]:
tsla.quarterly_financials

2021-12-31     2021-09-30  \
Research Development                      740000000.0    611000000.0   
Effect Of Accounting Charges                     None           None   
Income Before Tax                        2635000000.0   1882000000.0   
Minority Interest                        1394000000.0   1441000000.0   
Net Income                               2321000000.0   1618000000.0   
Selling General Administrative           1494000000.0    994000000.0   
Gross Profit                             4847000000.0   3660000000.0   
Ebit                                     2613000000.0   2004000000.0   
Operating Income                         2613000000.0   2004000000.0   
Other Operating Expenses                          NaN     51000000.0   
Interest Expense                          -70000000.0   -125000000.0   
Extraordinary Items                              None           None   
Non Recurring                                    None           None   
Other Items                                      None           None   
Income Tax Expense                        292000000.0    223000000.0   
Total Revenue                           17719000000.0  13757000000.0   
Total Operating Expenses                15106000000.0  11753000000.0   
Cost Of Revenue                         12872000000.0  10097000000.0   
Total Other Income Expense Net             22000000.0   -122000000.0   
Discontinued Operations                          None           None   
Net Income From Continuing Ops           2343000000.0   1659000000.0   
Net Income Applicable To Common Shares   2326000000.0   1618000000.0   

                                           2021-06-30     2021-03-31  
Research Development                      576000000.0    666000000.0  
Effect Of Accounting Charges                     None           None  
Income Before Tax                        1293000000.0    533000000.0  
Minority Interest                        1446000000.0   1448000000.0  
Net Income                               1142000000.0    438000000.0  
Selling General Administrative            973000000.0   1056000000.0  
Gross Profit                             2884000000.0   2215000000.0  
Ebit                                     1312000000.0    594000000.0  
Operating Income                         1312000000.0    594000000.0  
Other Operating Expenses                   23000000.0   -101000000.0  
Interest Expense                          -84000000.0    -79000000.0  
Extraordinary Items                              None           None  
Non Recurring                                    None           None  
Other Items                                      None           None  
Income Tax Expense                        115000000.0     69000000.0  
Total Revenue                           11958000000.0  10389000000.0  
Total Operating Expenses                10646000000.0   9795000000.0  
Cost Of Revenue                          9074000000.0   8174000000.0  
Total Other Income Expense Net            -19000000.0    -61000000.0  
Discontinued Operations                          None           None  
Net Income From Continuing Ops           1178000000.0    464000000.0  
Net Income Applicable To Common Shares   1142000000.0    438000000.0

In [20]:
tsla.cashflow

2021-12-31    2020-12-31  \
Investments                               -1.320000e+08 -1.320000e+08   
Change To Liabilities                      5.371000e+09  2.423000e+09   
Total Cashflows From Investing Activities -7.868000e+09 -3.132000e+09   
Net Borrowings                            -5.732000e+09 -2.488000e+09   
Total Cash From Financing Activities      -5.203000e+09  9.973000e+09   
Change To Operating Activities            -3.014000e+09 -1.165000e+09   
Issuance Of Stock                          7.070000e+08  1.268600e+10   
Net Income                                 5.519000e+09  7.210000e+08   
Change In Cash                            -1.757000e+09  1.311800e+10   
Effect Of Exchange Rate                   -1.830000e+08  3.340000e+08   
Total Cash From Operating Activities       1.149700e+10  5.943000e+09   
Depreciation                               2.810000e+09  2.322000e+09   
Other Cashflows From Investing Activities  6.000000e+06  1.230000e+08   
Change To Inventory                       -1.709000e+09 -4.220000e+08   
Change To Account Receivables             -1.300000e+08 -6.520000e+08   
Other Cashflows From Financing Activities -1.780000e+08 -2.250000e+08   
Change To Netincome                        2.650000e+09  2.716000e+09   
Capital Expenditures                      -8.014000e+09 -3.232000e+09   

                                             2019-12-31    2018-12-31  
Investments                               -1.320000e+08 -1.320000e+08  
Change To Liabilities                      1.447000e+09  2.203000e+09  
Total Cashflows From Investing Activities -1.436000e+09 -2.337000e+09  
Net Borrowings                             7.980000e+08  8.900000e+07  
Total Cash From Financing Activities       1.529000e+09  5.740000e+08  
Change To Operating Activities            -1.000000e+09 -6.250000e+08  
Issuance Of Stock                          1.111000e+09  2.960000e+08  
Net Income                                -8.620000e+08 -9.760000e+08  
Change In Cash                             2.506000e+09  3.120000e+08  
Effect Of Exchange Rate                    8.000000e+06 -2.300000e+07  
Total Cash From Operating Activities       2.405000e+09  2.098000e+09  
Depreciation                               2.092000e+09  1.888000e+09  
Other Cashflows From Investing Activities  4.600000e+07  4.600000e+07  
Change To Inventory                       -4.290000e+08 -1.023000e+09  
Change To Account Receivables             -3.670000e+08 -4.970000e+08  
Other Cashflows From Financing Activities -3.800000e+08  1.890000e+08  
Change To Netincome                        1.524000e+09  9.690000e+08  
Capital Expenditures                      -1.432000e+09 -2.319000e+09

In [21]:
tsla.quarterly_cashflow

2021-12-31    2021-09-30  \
Investments                               -1.020000e+08 -3.000000e+07   
Change To Liabilities                      2.155000e+09  1.345000e+09   
Total Cashflows From Investing Activities -1.916000e+09 -1.855000e+09   
Net Borrowings                            -1.456000e+09 -1.526000e+09   
Total Cash From Financing Activities      -1.257000e+09 -1.381000e+09   
Change To Operating Activities            -7.840000e+08 -9.930000e+08   
Issuance Of Stock                          2.620000e+08  1.920000e+08   
Net Income                                 2.321000e+09  1.618000e+09   
Change In Cash                             1.450000e+09 -1.310000e+08   
Effect Of Exchange Rate                    3.800000e+07 -4.200000e+07   
Total Cash From Operating Activities       4.585000e+09  3.147000e+09   
Depreciation                               7.470000e+08  7.610000e+08   
Change To Inventory                       -5.340000e+08 -4.880000e+08   
Change To Account Receivables              1.800000e+07  1.350000e+08   
Other Cashflows From Financing Activities -6.300000e+07 -4.700000e+07   
Change To Netincome                        6.620000e+08  7.690000e+08   
Capital Expenditures                      -1.814000e+09 -1.825000e+09   
Other Cashflows From Investing Activities           NaN           NaN   

                                             2021-06-30    2021-03-31  
Investments                               -3.000000e+07 -3.000000e+07  
Change To Liabilities                      1.037000e+09  8.340000e+08  
Total Cashflows From Investing Activities -1.515000e+09 -2.582000e+09  
Net Borrowings                            -1.588000e+09 -1.162000e+09  
Total Cash From Financing Activities      -1.549000e+09 -1.016000e+09  
Change To Operating Activities            -5.210000e+08 -7.160000e+08  
Issuance Of Stock                          7.000000e+07  1.830000e+08  
Net Income                                 1.142000e+09  4.380000e+08  
Change In Cash                            -8.980000e+08 -2.178000e+09  
Effect Of Exchange Rate                    4.200000e+07 -2.210000e+08  
Total Cash From Operating Activities       2.124000e+09  1.641000e+09  
Depreciation                               6.810000e+08  6.210000e+08  
Change To Inventory                       -5.810000e+08 -1.060000e+08  
Change To Account Receivables             -2.590000e+08 -2.400000e+07  
Other Cashflows From Financing Activities -3.100000e+07 -3.700000e+07  
Change To Netincome                        6.250000e+08  5.940000e+08  
Capital Expenditures                      -1.515000e+09 -2.860000e+09  
Other Cashflows From Investing Activities           NaN  6.000000e+06

In [22]:
tsla.earnings

Revenue    Earnings
Year                         
2018  21461000000  -976000000
2019  24578000000  -862000000
2020  31536000000   721000000
2021  53823000000  5519000000

In [23]:
tsla.quarterly_earnings

Revenue    Earnings
Quarter                         
1Q2021   10389000000   438000000
2Q2021   11958000000  1142000000
3Q2021   13757000000  1618000000
4Q2021   17719000000  2321000000

We can collect the above information on any security. With this data on our fingertips, how do we interpret the data?